In [1]:
# pandas를 불러옵니다. pandas는 데이터를 다루는데 유용한 라이브러리입니다.
import pandas as pd

# scikit-learn에서 train_test_split을 불러옵니다.
from sklearn.model_selection import train_test_split

# scikit-learn의 StandardScaler와 OneHotEncoder를 불러옵니다.
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Keras에서 Sequential, Dense, EarlyStopping을 불러옵니다.
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [3]:
# 데이터셋을 불러오기 위해 Pandas를 사용합니다. 데이터는 외부 URL에서 가져옵니다.
data = pd.read_csv('https://raw.githubusercontent.com/BigDeepData/2312_dl/main/data/Iris.csv')

# 데이터셋의 특성과 라벨을 분리합니다.
# 특성 데이터는 첫 번째부터 네 번째 컬럼을 사용합니다.
X = data.iloc[:, 1:5]  # 1부터 4까지의 컬럼 선택
# 라벨 데이터는 다섯 번째 컬럼을 사용합니다.
y = data.iloc[:, 5]    # 5번째 컬럼 선택 (라벨)

# 라벨을 원-핫 인코딩합니다.
encoder = OneHotEncoder()
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1)).toarray()

# 데이터를 정규화합니다. StandardScaler를 사용하여 평균이 0, 표준편차가 1이 되도록 스케일링합니다.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습 데이터와 테스트 데이터로 분리합니다. test_size는 테스트 데이터의 비율을 나타냅니다.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2)

In [4]:
# Sequential 모델을 생성합니다. Sequential 모델은 레이어를 순차적으로 쌓을 수 있는 모델입니다.
model = Sequential()

# 입력층을 추가합니다. input_dim은 입력 특성의 개수를 나타냅니다 (4개의 특성).
# 활성화 함수로 'relu'를 사용합니다.
model.add(Dense(10, input_dim=4, activation='relu'))  # 입력층

# 은닉층을 추가합니다. 10개의 뉴런을 가진 은닉층을 만듭니다.
# 활성화 함수로 'relu'를 사용합니다.
model.add(Dense(10, activation='relu'))               # 은닉층

# 출력층을 추가합니다. 출력층은 클래스의 개수와 같은 뉴런 수를 가지며,
# 다중 클래스 분류 문제이므로 활성화 함수로 'softmax'를 사용합니다.
model.add(Dense(3, activation='softmax'))             # 출력층

# 모델의 구조를 요약하여 출력합니다.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 193 (772.00 Byte)
Trainable params: 193 (772.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# 모델을 컴파일합니다. 컴파일 단계에서는 모델의 학습 방법을 설정합니다.

# 손실 함수(loss function)를 'categorical_crossentropy'로 설정합니다.
# 이것은 다중 클래스 분류 문제에 적합한 손실 함수입니다.
# 다중 클래스 분류에서는 각 클래스에 대한 확률 분포의 차이를 최소화하는 것이 목표입니다.
model.compile(loss='categorical_crossentropy',

              # 최적화 알고리즘으로 'adam'을 사용합니다.
              # 'adam'은 효과적으로 경사 하강법을 수행하는 알고리즘 중 하나입니다.
              optimizer='adam',

              # 모델을 평가할 때 사용할 지표를 설정합니다.
              # 'accuracy'는 정확도를 나타내며, 올바르게 분류된 샘플의 비율입니다.
              metrics=['accuracy'])

In [6]:
# 조기 종료 콜백(EarlyStopping)을 설정합니다.

# monitor 매개변수는 모니터링할 지표를 선택합니다.
# 여기서 'val_loss'를 선택하면 검증 데이터셋의 손실 함수 값이 모니터링됩니다.
early_stopping = EarlyStopping(

    # 모니터링할 지표로 'val_loss'를 선택합니다.
    monitor='val_loss',

    # 'patience' 매개변수는 조기 종료를 얼마나 오랫동안 지켜볼지를 나타냅니다.
    # 예를 들어, 'patience=10'은 10 에포크 동안 검증 손실이 개선되지 않으면 학습을 조기 종료합니다.
    patience=10
)

In [7]:
# 모델을 학습합니다.

# X_train: 학습 데이터의 입력 특성
# y_train: 학습 데이터의 라벨 (원-핫 인코딩된 형태)
# epochs: 학습을 몇 번 반복할지를 나타냅니다. 여기서는 200번을 설정했습니다.
# batch_size: 한 번에 학습할 샘플의 개수를 나타냅니다. 여기서는 10으로 설정했습니다.
# validation_split: 학습 데이터 중 일부를 검증(validation) 데이터로 사용할 비율을 나타냅니다.
# 여기서는 0.2로 설정했으므로 학습 데이터의 20%를 검증 데이터로 사용합니다.
# callbacks: 모델 학습 중에 사용할 콜백 함수를 설정합니다. 여기서는 조기 종료 콜백을 사용합니다.

model.fit(
    X_train,                    # 학습 데이터의 입력 특성
    y_train,                    # 학습 데이터의 라벨
    epochs=200,                 # 학습 반복 횟수
    batch_size=10,              # 한 번에 학습할 샘플 개수
    validation_split=0.2,       # 검증 데이터의 비율
    callbacks=[early_stopping]  # 조기 종료 콜백 사용
)


Epoch 1/200
10/10 [==============================] - 2s 27ms/step - loss: 1.1959 - accuracy: 0.4583 - val_loss: 1.2940 - val_accuracy: 0.4167
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 1.1309 - accuracy: 0.5938 - val_loss: 1.2353 - val_accuracy: 0.4583
Epoch 3/200
10/10 [==============================] - 0s 5ms/step - loss: 1.0739 - accuracy: 0.6042 - val_loss: 1.1782 - val_accuracy: 0.5000
Epoch 4/200
10/10 [==============================] - 0s 8ms/step - loss: 1.0174 - accuracy: 0.6250 - val_loss: 1.1266 - val_accuracy: 0.5417
Epoch 5/200
10/10 [==============================] - 0s 5ms/step - loss: 0.9640 - accuracy: 0.6562 - val_loss: 1.0819 - val_accuracy: 0.5417
Epoch 6/200
10/10 [==============================] - 0s 5ms/step - loss: 0.9103 - accuracy: 0.6667 - val_loss: 1.0368 - val_accuracy: 0.5417
Epoch 7/200
10/10 [==============================] - 0s 7ms/step - loss: 0.8621 - accuracy: 0.6667 - val_loss: 0.9943 - val_accuracy: 0.5417
Epoch 8/200


In [8]:
# 모델을 평가합니다.

# model.evaluate() 함수를 사용하여 모델을 평가합니다.
# X_test: 테스트 데이터의 입력 특성
# y_test: 테스트 데이터의 실제 라벨 (원-핫 인코딩된 형태)

# 모델의 평가 결과는 _, accuracy 변수에 저장됩니다.
# 여기서 '_'는 무시할 값으로, 손실(loss)을 나타내는 값이지만 이 코드에서는 사용하지 않습니다.

_, accuracy = model.evaluate(X_test, y_test)

# 정확도를 출력합니다.
# accuracy 변수에 저장된 값은 정확도를 나타냅니다.
# 정확도는 올바르게 분류된 샘플의 비율을 나타냅니다.
print(f'Accuracy: {accuracy*100:.2f}%')

1/1 [==============================] - 0s 33ms/step - loss: 0.1225 - accuracy: 0.9333
Accuracy: 93.33%
